In [22]:
import pandas as pd
import pandasql as ps

In [2]:
customers = pd.read_csv("customers.csv")
geolocation = pd.read_csv("geolocation.csv")
order_items = pd.read_csv("order_items.csv")
payments = pd.read_csv("payments.csv")
reviews = pd.read_csv("order_reviews.csv", encoding='latin-1')
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
sellers = pd.read_csv("sellers.csv")

In [5]:
#1.1

print(customers.info())
print("--------------------------------------------------------------------------------------------------------")
print(geolocation.info())
print("--------------------------------------------------------------------------------------------------------")
print(order_items.info())
print("--------------------------------------------------------------------------------------------------------")
print(payments.info())
print("--------------------------------------------------------------------------------------------------------")
print(reviews.info())
print("--------------------------------------------------------------------------------------------------------")
print(orders.info())
print("--------------------------------------------------------------------------------------------------------")
print(products.info())
print("--------------------------------------------------------------------------------------------------------")
print(sellers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
--------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2

In [8]:
#1.2

q12= '''select min(order_purchase_timestamp) as start_date, max(order_purchase_timestamp) as last_date from orders'''
print(ps.sqldf(q12))

            start_date            last_date
0  2016-09-04 21:15:19  2018-10-17 17:30:18


In [13]:
#1.3

q13 = '''select distinct c.customer_city,c.customer_state 
        from orders as o join customers as c on o.customer_id=c.customer_id '''
print(ps.sqldf(q13))


                customer_city customer_state
0                   sao paulo             SP
1                   barreiras             BA
2                  vianopolis             GO
3     sao goncalo do amarante             RN
4                 santo andre             SP
...                       ...            ...
4305                   pacuja             CE
4306                umburanas             BA
4307          messias targino             RN
4308         campo do tenente             PR
4309              nova vicosa             BA

[4310 rows x 2 columns]


In [24]:
#2.1

q21 = '''select strftime('%Y',order_purchase_timestamp) as year,
        strftime('%m',order_purchase_timestamp) as month,count(order_id) as sales_per_month 
        from orders group by year,month
        order by year,month'''
print(ps.sqldf(q21))

    year month  sales_per_month
0   2016    09                4
1   2016    10              324
2   2016    12                1
3   2017    01              800
4   2017    02             1780
5   2017    03             2682
6   2017    04             2404
7   2017    05             3700
8   2017    06             3245
9   2017    07             4026
10  2017    08             4331
11  2017    09             4285
12  2017    10             4631
13  2017    11             7544
14  2017    12             5673
15  2018    01             7269
16  2018    02             6728
17  2018    03             7211
18  2018    04             6939
19  2018    05             6873
20  2018    06             6167
21  2018    07             6292
22  2018    08             6512
23  2018    09               16
24  2018    10                4


In [34]:
#2.2

q22 = '''select CASE 
            when strftime('%H',order_purchase_timestamp) between "02" and "05" then "DAWN"
            when strftime('%H',order_purchase_timestamp) between "05" and "12" then "Morning"
            when strftime('%H',order_purchase_timestamp) between "12" and "17" then "AfterNoon"
            when strftime('%H',order_purchase_timestamp) between "17" and "21" then "Evening"
            else "Night" 
            End as Hour,count(order_id) as Orders from orders group by Hour
        
        '''

print(ps.sqldf(q22))

        Hour  Orders
0  AfterNoon   32366
1       DAWN    1176
2    Evening   24161
3    Morning   28235
4      Night   13503


In [49]:
#3.1

q31 = '''select strftime('%m',order_purchase_timestamp) as month,c.customer_state as State,count(order_id) as sales_per_month 
        from customers as c join orders as o on o.customer_id=c.customer_id 
        group by month, State order by month'''
print(ps.sqldf(q31))

    month State  sales_per_month
0      01    AC                8
1      01    AL               39
2      01    AM               12
3      01    AP               11
4      01    BA              264
..    ...   ...              ...
317    12    RS              283
318    12    SC              193
319    12    SE               20
320    12    SP             2357
321    12    TO               14

[322 rows x 3 columns]


In [48]:
#3.2

q32 = '''select customer_state as State, count(customer_id) as Number_of_Customer
        from customers group by State
        '''
print(ps.sqldf(q32))

   State  Number_of_Customer
0     AC                  81
1     AL                 413
2     AM                 148
3     AP                  68
4     BA                3380
5     CE                1336
6     DF                2140
7     ES                2033
8     GO                2020
9     MA                 747
10    MG               11635
11    MS                 715
12    MT                 907
13    PA                 975
14    PB                 536
15    PE                1652
16    PI                 495
17    PR                5045
18    RJ               12852
19    RN                 485
20    RO                 253
21    RR                  46
22    RS                5466
23    SC                3637
24    SE                 350
25    SP               41746
26    TO                 280


In [6]:
#4.1

q41 = '''
        with base as(
        select sum(p.payment_value) as Payments,strftime('%Y',o.order_purchase_timestamp) as year
        from orders o join payments p on o.order_id = p.order_id
        where strftime('%m',o.order_purchase_timestamp) BETWEEN "01" AND "08" 
        group by year),
        base_2 as (
        select *, lag(payments,1 ) over (order by year asc) as Prev_Year_Payments from base
        ),
        base_3 as (
        select *, round((Payments-Prev_Year_Payments)/ Prev_Year_Payments *100,2) as Percentage_change from base_2
        )
        select * from base_3 where Prev_Year_Payments is not null
        '''
print(ps.sqldf(q41))

     Payments  year  Prev_Year_Payments  Percentage_change
0  8694733.84  2018          3669022.12             136.98


In [11]:
#4.2

q42 = '''
        select c.customer_state as State, sum(oi.price) as Price, sum(oi.freight_value) as Freight_Value,
        avg(oi.price) as Avg_Price, avg(oi.freight_value) as Avg_FreightValue
        from order_items as oi
        join orders as o on oi.order_id= o.order_id
        join customers as c on o.customer_id = c.customer_id
        group by State
    '''
print(ps.sqldf(q42))

   State         Price  Freight_Value   Avg_Price  Avg_FreightValue
0     AC  1.598295e+04        3686.75  173.727717         40.073370
1     AL  8.031481e+04       15914.59  180.889212         35.843671
2     AM  2.235684e+04        5478.89  135.496000         33.205394
3     AP  1.347430e+04        2788.50  164.320732         34.006098
4     BA  5.113500e+05      100156.68  134.601208         26.363959
5     CE  2.272547e+05       48351.59  153.758261         32.714202
6     DF  3.026039e+05       50625.50  125.770549         21.041355
7     ES  2.750373e+05       49764.60  121.913701         22.058777
8     GO  2.945919e+05       53114.98  126.271732         22.766815
9     MA  1.196482e+05       31523.77  145.204150         38.257002
10    MG  1.585308e+06      270853.46  120.748574         20.630167
11    MS  1.168126e+05       19144.03  142.628376         23.374884
12    MT  1.564535e+05       29715.43  148.297185         28.166284
13    PA  1.789478e+05       38699.30  165.69241

In [28]:
#5.1

q51 = """SELECT strftime('%d',order_estimated_delivery_date)-strftime('%d',order_purchase_timestamp) as Estimation_Delivery_Time,
        strftime('%d',order_delivered_carrier_date)-strftime('%d',order_purchase_timestamp) as Delivered_Time
        FROM orders"""
print(ps.sqldf(q51))

       Estimation_Delivery_Time  Delivered_Time
0                            16             2.0
1                           -11             2.0
2                            -4             0.0
3                            -3             4.0
4                            13             1.0
...                         ...             ...
99436                        19             1.0
99437                        -4             1.0
99438                         0             1.0
99439                         7             4.0
99440                        -5             1.0

[99441 rows x 2 columns]


In [33]:
#5.2

q52 = '''SELECT Cast ((JulianDay(order_delivered_customer_date) - JulianDay(order_purchase_timestamp)) As Integer) as time_to_delivery,
        Cast ((JulianDay(order_estimated_delivery_date) - JulianDay(order_delivered_customer_date)) As Integer) as diff_estimated_delivery
        FROM orders'''
print(ps.sqldf(q52))

       time_to_delivery  diff_estimated_delivery
0                   8.0                      7.0
1                  13.0                      5.0
2                   9.0                     17.0
3                  13.0                     12.0
4                   2.0                      9.0
...                 ...                      ...
99436               8.0                     10.0
99437              22.0                      1.0
99438              24.0                      5.0
99439              17.0                     20.0
99440               7.0                     17.0

[99441 rows x 2 columns]


In [36]:
#5.3

q53 = '''SELECT customer_state as State,AVG(ot.freight_value) as avg_freight_value,
        AVG(Cast ((JulianDay(od.order_delivered_customer_date) - JulianDay(od.order_purchase_timestamp)) As Integer)) as avg_time_to_delivery,
        AVG(Cast ((JulianDay(od.order_estimated_delivery_date) - JulianDay(od.order_delivered_customer_date)) As Integer)) as avg_diff_estimated_delivery
        FROM order_items  ot
        join orders od on od.order_id=ot.order_id
        join customers cs on cs.customer_id=od.customer_id
        group by State
'''
print(ps.sqldf(q53))

   State  avg_freight_value  avg_time_to_delivery  avg_diff_estimated_delivery
0     AC          40.073370             20.329670                    20.010989
1     AL          35.843671             23.992974                     7.976581
2     AM          33.205394             25.963190                    18.975460
3     AP          34.006098             27.753086                    17.444444
4     BA          26.363959             18.774640                    10.119468
5     CE          32.714202             20.537167                    10.256662
6     DF          21.041355             12.501486                    11.274735
7     ES          22.058777             15.192809                     9.768539
8     GO          22.766815             14.948177                    11.372859
9     MA          38.257002             21.203750                     9.110000
10    MG          20.630167             11.515522                    12.397151
11    MS          23.374884             15.107275   

In [41]:
#5.5
print("Highest")
q551 = '''SELECT customer_state,
        AVG(Cast ((JulianDay(od.order_estimated_delivery_date) - JulianDay(od.order_delivered_customer_date)) As Integer)) as avg_diff_estimated_delivery
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_diff_estimated_delivery desc
        limit 5
        '''
print(ps.sqldf(q551))

print("Lowest")

q552 = '''SELECT customer_state,
        AVG(Cast ((JulianDay(od.order_estimated_delivery_date) - JulianDay(od.order_delivered_customer_date)) As Integer)) as avg_diff_estimated_delivery
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_diff_estimated_delivery asc
        limit 5
        '''
print(ps.sqldf(q552))


Highest
  customer_state  avg_diff_estimated_delivery
0             AC                    20.010989
1             RO                    19.080586
2             AM                    18.975460
3             AP                    17.444444
4             RR                    17.434783
Lowest
  customer_state  avg_diff_estimated_delivery
0             AL                     7.976581
1             MA                     9.110000
2             SE                     9.165333
3             ES                     9.768539
4             BA                    10.119468


In [44]:
#5.6
print("Highest")
q561 = """SELECT customer_state,
        AVG(Cast ((JulianDay(od.order_delivered_customer_date) - JulianDay(od.order_purchase_timestamp)) As Integer)) as avg_time_to_delivery
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_time_to_delivery desc
        limit 5
"""
print(ps.sqldf(q561))
print("Lowest")

q562 = """SELECT customer_state,
        AVG(Cast ((JulianDay(od.order_delivered_customer_date) - JulianDay(od.order_purchase_timestamp)) As Integer)) as avg_time_to_delivery
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_time_to_delivery asc
        limit 5
"""
print(ps.sqldf(q562))



Highest
  customer_state  avg_time_to_delivery
0             RR             27.826087
1             AP             27.753086
2             AM             25.963190
3             AL             23.992974
4             PA             23.301708
Lowest
  customer_state  avg_time_to_delivery
0             SP              8.259609
1             PR             11.480793
2             MG             11.515522
3             DF             12.501486
4             SC             14.520986


In [45]:
#5.7
print("Highest")
q571 = """SELECT customer_state, freight_value, AVG(ot.freight_value) as avg_freight_value
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_freight_value, freight_value desc
        limit 5
"""
print(ps.sqldf(q571))
print('Lowest')
q572 = """SELECT customer_state, freight_value, AVG(ot.freight_value) as avg_freight_value
        FROM order_items  ot
        inner join orders od on od.order_id=ot.order_id
        inner join customers cs on cs.customer_id=od.customer_id
        group by customer_state
        order by avg_freight_value, freight_value asc
        limit 5
"""
print(ps.sqldf(q572))

Highest
  customer_state  freight_value  avg_freight_value
0             SP          19.93          15.147275
1             PR          15.10          20.531652
2             MG          17.87          20.630167
3             RJ          13.29          20.960924
4             DF          21.75          21.041355
Lowest
  customer_state  freight_value  avg_freight_value
0             SP          19.93          15.147275
1             PR          15.10          20.531652
2             MG          17.87          20.630167
3             RJ          13.29          20.960924
4             DF          21.75          21.041355


In [49]:
#6.1

q61 = '''
        select strftime('%m',o.order_purchase_timestamp) as Month,p.payment_type, count(p.payment_type) as Payment_Type from
        orders as o join payments as p on o.order_id = p.order_id
        group by Month,Payment_Type
    '''
print(ps.sqldf(q61))

   Month payment_type  Payment_Type
0     01          UPI          1715
1     01  credit_card          6103
2     01   debit_card           118
3     01      voucher           477
4     02          UPI          1723
5     02  credit_card          6609
6     02   debit_card            82
7     02      voucher           424
8     03          UPI          1942
9     03  credit_card          7707
10    03   debit_card           109
11    03      voucher           591
12    04          UPI          1783
13    04  credit_card          7301
14    04   debit_card           124
15    04      voucher           572
16    05          UPI          2035
17    05  credit_card          8350
18    05   debit_card            81
19    05      voucher           613
20    06          UPI          1807
21    06  credit_card          7276
22    06   debit_card           209
23    06      voucher           563
24    07          UPI          2074
25    07  credit_card          7841
26    07   debit_card       

In [51]:
#6.2

q62 = '''
        select payment_installments, count(payment_installments) as count from payments group by payment_installments
        '''
print(ps.sqldf(q62))

    payment_installments  count
0                      0      2
1                      1  52546
2                      2  12413
3                      3  10461
4                      4   7098
5                      5   5239
6                      6   3920
7                      7   1626
8                      8   4268
9                      9    644
10                    10   5328
11                    11     23
12                    12    133
13                    13     16
14                    14     15
15                    15     74
16                    16      5
17                    17      8
18                    18     27
19                    20     17
20                    21      3
21                    22      1
22                    23      1
23                    24     18


In [50]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


## Actionable Insights :

- There is an increase in trend of orders month over month.
- Sales reaches peak on November, December, and January during festival seasons.
- Most preferred time for customers placing their orders are during afternoon and night.
- Around 77% of customers are from five states out of 27 states in Brazil with one state alone contributing 42% of the customer base.
- Around an average of 60% jump on cost spent through e-commerce each month from 2017 to 2018 (Jan - Aug).
- Freight values are lower in states and time taken for delivery is less where customer distribution is high.
- Most preferred modes of payment is through credit cards followed by UPI, Debit cards.
- Most customers prefer 1 â€“ 5 installment payments where almost half of the customers prefer single installment.


## Recommendations:

- During peak seasons of November, December and January; advertising and providing discounts.
- Based on the purchase and search history of customers sending notifications, mails with various discounts on those products.
- From the data that around 2000 vouchers used each during the year 2017 and 2018, clubbing with sports and important events with voucher opportunities to attract customers.
- Targeting states with ads in lower customer distribution states.
- Pushing notifications in apps, websites and social media during the afternoon and night time where customers tend to place most of their orders.
- Clubbing with financial companies especially credit card companies by providing various EMI options, offers.
- Adding new sellers in lower customer distribution areas in order to reduce the freight value and time taken for delivery and therefore enhancing the customer experiences.